In [89]:
import pandas as pd
import ast

data = pd.read_csv('/home/keonwoo/anaconda3/envs/bgmRS/data/lead_newspaper.csv')

data['text'] = data['text'].apply(lambda x: ast.literal_eval(x))
data['text'] = data['text'].apply(lambda x: " ".join(x))

In [90]:
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import torch
import pandas as pd
import numpy as np

ckpt_path = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
sbert_tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
sbert_model = AutoModel.from_pretrained(ckpt_path, output_hidden_states=True)

In [97]:
a = [0,1,2,3,4]
a[:3]

[0, 1, 2]

In [101]:
def encode_sentence(tokenizer, model, text):

    
    if len(tokenizer.tokenize(text)) >= 510:
        text = text[:510]
        encoded_input = tokenizer(text, return_tensors='pt')

    else:
        encoded_input = tokenizer(text, return_tensors='pt')

    model.eval()
    
    with torch.no_grad():
        outputs = model(**encoded_input)
        
        o1, o2, o3 = outputs['last_hidden_state'], outputs['pooler_output'], outputs['hidden_states']
        bertcls  = o2.squeeze().detach().numpy()
        o1 = o1.squeeze().detach().numpy()
        cls_token = o1[0]
    

    return cls_token

In [102]:
label_dict = {
    '희망적인' : '만족스러운',  # 소분류 
    '감동적인' : '감사하는', # 소분류
    '무서운' : '두려운',  # 불안 -> 소분류
    '로맨틱한' : '느긋', # 소분류
    '평화로운' : '편안한', # 소분류 good
    '멋진' : '자신하는', # 소분류
    '우스운' : '흥분', # 소분류
    '신나는' : '신이 난', # 소분류
    '쿨한' : '안도' # 소분류
}

tag_emb = []

for k in label_dict.keys():
    tag_emb.append(encode_sentence(sbert_tokenizer, sbert_model, k))


In [129]:
from sklearn.metrics.pairwise import cosine_similarity

label_list = []
for text in data['text']:

    kk = encode_sentence(sbert_tokenizer, sbert_model, text)
    cos_sim = cosine_similarity([kk], tag_emb)
    if np.max(cos_sim) < 0.25:
        label = None
    else:
        arg = np.argmax(cos_sim)
        label = list(label_dict.keys())[arg]
    label_list.append(label)

In [130]:
news_data = pd.concat([pd.DataFrame(label_list,columns=['label']),data['text']],axis=1)

In [131]:
sum(news_data['label'].isnull())

1930

In [134]:
news_data[news_data['label'] != None]

,label,text


In [139]:
news_data.iloc[124]['text']

'“올해 총물동량 3억톤, 컨테이너 245만TEU 목표” 이성훈 sinawi@hanmail.net 석유화학부두 저장시설 설치, 신사업 적극 추진 지난해 3월 17일 취임한 방희석 여수광양항만공사 사장이 취임 1주년을 맞았다. 방 사장은 취임 후 여수광양항 물동량 증대와 공사 부채 해소를 통한 재무건전성 강화, 미래성장 신사업 발굴 등 지속가능한 경영과 경쟁력 강화에 역점을 뒀다. 또한 공기업으로서의 공적기능 확충과 정부 정책의 적극적인 이행, 동반성장 및 사회공헌 활동 등 국민으로부터 신뢰받는 공기업의 위상을 제고하는데 앞장 서 왔다는 평가다. 방 사장은“광양항 컨테이너 물동량 증대뿐만 아니라 철강, 석유화학, 자동차 등 글로벌 복합물류항만으로 성장시키기 위한 다양한 제도를 지속적으로 강화해 나갈 것”이라며“차질없는 미래 신성장 사업 추진 등 광양항 활성화에 최선을 다할 것”이라고 말했다. 방 사장은 올해 추진사업으로“컨테이너 물동량 증가와 연계된 인센티브 제도를 강화할 것”이라며“총물동량 3억톤, 컨테이너물동량 245만TEU, 매출액 1163억원 등 경영 목표를 달성해 나가겠다”고 다짐했다. 이어“미래 신성장 사업을 차질없이 추진하는 한편, 광양항 활성화를 위한 다양한 사업을 추진할 계획이다”고 밝혔다. 이를 살펴보면 해양산업클러스터 구축을 위해 항만물류 R&D 관련 기업의 수요를 조사하고 유관기관과 공동으로 기업 및 투자 유치를 추진할 방침이다. 석유화학부두 저장시설 설치사업은 연내에 착공하고 이를 위한 저장시설 운영사 모집 및 선정, 용역결과 기반 기본 및 실시설계에 들어갈 계획이다. 리스컨테이너 장치장 확보사업도 올 상반기 시설 준공 및 운영 개시를 목표로 세척 및 세척시설 설치공사에 들어갈 예정이다. 방 사장은“자동차 환적중심기지 육성 사업으로 자동차부두 생산성 제고 및 자동차 환적 물량 증대에 대응하기 위한 초대형 자동차전용부두를 확보할 것”이라며“포장공사가 완료된 광양항 컨테이너부두 19, 20번 선석은 자동차부두로 운영해 일시장치능력 4만대, 연